##Set-up

In [1]:
# installs
! pip install -q pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 7.0 MB/s eta 0:00:00


In [1]:
# imports - this will take a few moments
import pandas as pd
import numpy as np
import duckdb

import duckdb
import langchain
from pinecone import Pinecone, ServerlessSpec

import os
import uuid

from google.colab import userdata

import vertexai
from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel
from vertexai.generative_models import GenerativeModel, ChatSession
from vertexai.generative_models import Part

from sentence_transformers import SentenceTransformer



In [2]:
# easiest path, auth with your BU account that you are using on GCP
from google.colab import auth
auth.authenticate_user()

In [3]:
# if you have your service account
# first upload to the main folder on the left
# right click the json service account file and get the path
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/ba882-team8-fall24-4396508fd779.json'

In [4]:
project_id = "ba882-team8-fall24"   # <--- your project, mine is btibert-ba882-fall24
region_id = "us-central1"

vertexai.init(project=project_id, location=region_id)

model = GenerativeModel("gemini-1.5-pro-001")

In [5]:
# lets define a prompt
prompt = """
Help me learn about Generative AI on VertexAI with python.
"""

response = model.generate_content(prompt)
print(response.text)

## Diving into Generative AI on Vertex AI with Python

Vertex AI is Google Cloud's unified machine learning platform that provides tools for building, training, and deploying machine learning models, including those powered by Generative AI. Here's a breakdown of key aspects and how to get started with Python:

**1. What is Generative AI?**

Generative AI refers to a class of machine learning models that can create new data instances, like text, images, audio, code, and more, that resemble the training data.  Examples include:

* **Text Generation:** Language models like GPT-3 can generate human-quality text, write stories, translate languages, and summarize documents.
* **Image Generation:** Models like DALL-E and Stable Diffusion can generate realistic images from text descriptions (text-to-image).
* **Code Generation:**  Models like Codex can generate code in various programming languages from natural language descriptions.

**2. Generative AI on Vertex AI**

Vertex AI provides a ma

In [ ]:
# what do we have for the response
response


candidates {
  content {
    role: "model"
    parts {
      text: "## Generative AI on Vertex AI with Python: A Comprehensive Guide\n\nGenerative AI on Vertex AI empowers you to build and deploy powerful AI applications using Google\'s cutting-edge technology. This guide provides a structured approach to mastering this exciting field:\n\n**1. Understanding the Basics:**\n\n* **What is Generative AI?** It\'s a branch of AI focused on creating new content, like text, images, audio, etc., rather than just analyzing existing data. \n* **Vertex AI:** Google Cloud\'s unified platform for building and deploying machine learning models, including generative models.\n\n**2. Key Generative AI Services on Vertex AI:**\n\n* **Generative Language Models (PaLM 2):** Foundation for text generation, summarization, question answering, and more. Accessible through Vertex AI PaLM 2 API.\n* **Imagen:** Text-to-image generation API for creating realistic and imaginative visuals.\n* **Codey:** Family of co

##Database creation

In [6]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Define your BigQuery table
project_id = "ba882-team8-fall24"  # Replace with your GCP project ID
dataset_id = "mbta_dataset"  # Replace with your dataset ID
table_id = "joined_prediction"      # Replace with your table ID

query = f"""
    SELECT *
    FROM `{project_id}.{dataset_id}.{table_id}`
"""

# Query BigQuery and load the data into a pandas DataFrame
query_job = client.query(query)
df = query_job.result().to_dataframe()

# Initialize DuckDB and create a database
db = duckdb.connect("mbta.duckdb")  # Creates a new DuckDB file

# Load the DataFrame into DuckDB as a table
db.execute("CREATE TABLE mbta AS SELECT * FROM df")

# Verify the table exists in DuckDB
result = db.execute("SHOW TABLES").fetchall()
print(result)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[('mbta',)]


In [7]:
db.sql("show tables")

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ mbta    │
└─────────┘

In [8]:
mbta = db.sql("select * from mbta;").df()
mbta.shape

(259199, 30)

In [9]:
mbta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259199 entries, 0 to 259198
Data columns (total 30 columns):
 #   Column                 Non-Null Count   Dtype                  
---  ------                 --------------   -----                  
 0   prediction_id          259199 non-null  object                 
 1   schedule_id            259199 non-null  object                 
 2   arrival_time_p         259199 non-null  datetime64[us, Etc/UTC]
 3   arrival_time_s         259199 non-null  datetime64[us, Etc/UTC]
 4   arrival_uncertainty    21827 non-null   float64                
 5   departure_time_p       259199 non-null  datetime64[us, Etc/UTC]
 6   departure_time_s       259199 non-null  datetime64[us, Etc/UTC]
 7   departure_uncertainty  21827 non-null   float64                
 8   direction_id_p         259199 non-null  int64                  
 9   direction_id_s         259199 non-null  int64                  
 10  last_trip              259199 non-null  bool            

## Pinecone


In [8]:
# I added the Pinecone API key to the secrets here in colab
# toggle on the switch to allow access within this notebook

#pinecone_token = userdata.get('PINECONE_BU')

pc = Pinecone(api_key="pcsk_4kKuo8_KkUeBkRBCXMTksnXBFDVf8ph7ro9nfN8z5N8qU3iLRXK2HNqZMiCAarvFoZiwm4")

In [9]:
# you likely wont have any, but I already have some pipelines running
# as well as for sandboxing ideas

pc.list_indexes()

[
    {
        "name": "mbta",
        "dimension": 385,
        "metric": "cosine",
        "host": "mbta-k167d5c.svc.gcp-us-central1-4a9f.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "gcp",
                "region": "us-central1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    },
    {
        "name": "ba882-rag",
        "dimension": 768,
        "metric": "cosine",
        "host": "ba882-rag-k167d5c.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    },
    {
        "name": "sample-movies",
        "dimension": 1024,
        "metric": "cosine",
        "host": "sample-movies-k167d5c

In [18]:
from tqdm import tqdm

# Initialize the embedding model
sent_trans = SentenceTransformer("all-MiniLM-L6-v2")

# Function to process a batch of rows
def process_batch(batch):
    return [sent_trans.encode(f"{row['day']} {row['time_of_the_day']}") for _, row in batch.iterrows()]

# Split data into batches
batch_size = 1000  # Adjust batch size as per available resources
batches = [mbta[i:i + batch_size] for i in range(0, len(mbta), batch_size)]

# Process batches
embeddings = []
for batch in tqdm(batches):
    embeddings.extend(process_batch(batch))

mbta['text_embedding'] = embeddings

100%|██████████| 260/260 [28:25<00:00,  6.56s/it]


In [16]:
from sklearn.preprocessing import MinMaxScaler

# Normalize numeric features (e.g., 'delay')
scaler = MinMaxScaler()
mbta['normalized_numeric'] = list(scaler.fit_transform(mbta[['delay']]))

In [19]:
# Combine embeddings (text + numeric)
mbta['combined_vector'] = mbta.apply(
    lambda row: np.concatenate([row['text_embedding'], row['normalized_numeric']]), axis=1
)

In [29]:
# a convention is to create the index if pinecone doesn't have it

index_name = "mbta"

dimension = len(mbta['combined_vector'].iloc[0])

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='gcp', # gcp <- not part of free
            region='us-central1' # us-central1 <- not part of free
        )
    )

In [10]:
# Connect to the index
index_name = "mbta"

index = pc.Index(index_name)

In [11]:
# get the stats
index.describe_index_stats()

{'dimension': 385,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 259199}},
 'total_vector_count': 259199}

In [34]:
from tqdm import tqdm

# Define batch size
batch_size = 1000  # Adjust this based on your system's capabilities

# Function to process and upsert a batch
def upsert_batch(index, batch):
    vectors = [
        {
            "id": str(idx),  # Unique ID
            "values": row['combined_vector'],  # Vector to upsert
            "metadata": {
                "day": row['day'],  # Supported fields only
                "time_of_the_day": row['time_of_the_day'],  # Include specific fields
                "delay": row['delay']  # Numeric field
            }
        }
        for idx, row in batch.iterrows()
    ]
    index.upsert(vectors)

# Process data in batches with progress tracking
for i in tqdm(range(0, len(mbta), batch_size), desc="Upserting vectors"):
    batch = mbta.iloc[i:i + batch_size]
    upsert_batch(index, batch)

Upserting vectors: 100%|██████████| 260/260 [11:53<00:00,  2.74s/it]


##Using vectorized data

In [6]:
# Initialize the embedding model
sent_trans = SentenceTransformer("all-MiniLM-L6-v2")

###User Input

In [12]:
# User's input
user_query = "What delays are typical on Monday night?"

# Convert query to embedding
query_vector = sent_trans.encode(user_query).tolist()
query_vector.append(0.0)

# Query Pinecone
response = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)

# Extract metadata
results = [
    {
        "day": match['metadata']['day'],
        "time_of_the_day": match['metadata']['time_of_the_day'],
        "delay": match['metadata']['delay'],
        "score": match['score']
    }
    for match in response['matches']
]

###LLM Response

In [20]:
# Format the context for Gemini
context = "\n".join(
    [
        f"Day: {entry['day']}, Time: {entry['time_of_the_day']}, Delay: {entry['delay']} minutes."
        for entry in results
    ]
)

# Generate a prompt for Gemini
prompt = f"""
The user asked: {user_query}
Here are some relevant data points:
{context}

Please interpret the data and provide a summary in a conversational tone.
"""

# Use Gemini to generate a response
response = model.generate_content(prompt)
print(response.text)

Hmm, this data is a bit strange! 

It looks like there was a HUGE negative delay on Tuesday evening.  A negative delay means things were actually really early!  It's unlikely that something would be over 20 hours early four times in a row. There might be an error with how this data was recorded. 

As for Monday night, there's only one data point showing a 16-minute delay.  

Overall, we can't really say what delays are typical on Monday nights based on this information. We need more accurate data about Monday nights specifically to draw any conclusions! 



##More Examples

In [27]:
# User's input
user_query = "What are the typical delays on Saturday nights?"

# Convert query to embedding
query_vector = sent_trans.encode(user_query).tolist()
query_vector.append(0.0)

# Query Pinecone
response = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)

# Extract metadata
results = [
    {
        "day": match['metadata']['day'],
        "time_of_the_day": match['metadata']['time_of_the_day'],
        "delay": match['metadata']['delay'],
        "score": match['score']
    }
    for match in response['matches']
]


# Format the context for Gemini
context = "\n".join(
    [
        f"Day: {entry['day']}, Time: {entry['time_of_the_day']}, Delay: {entry['delay']} minutes."
        for entry in results
    ]
)

# Generate a prompt for Gemini
prompt = f"""
The user asked: {user_query}
Here are some relevant data points:
{context}

Please interpret the data and provide a summary in a conversational tone.
"""

# Use Gemini to generate a response
response = model.generate_content(prompt)
print(response.text)

It looks like things run pretty smoothly on Saturday nights!  Those negative delays actually mean that things are arriving early by about 25-40 minutes on average.  Maybe everyone is just eager to get the weekend started! 🎉 



In [28]:
# User's input
user_query = "Why might delays be longer on Wednesday afternoons?"

# Convert query to embedding
query_vector = sent_trans.encode(user_query).tolist()
query_vector.append(0.0)

# Query Pinecone
response = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)

# Extract metadata
results = [
    {
        "day": match['metadata']['day'],
        "time_of_the_day": match['metadata']['time_of_the_day'],
        "delay": match['metadata']['delay'],
        "score": match['score']
    }
    for match in response['matches']
]


# Format the context for Gemini
context = "\n".join(
    [
        f"Day: {entry['day']}, Time: {entry['time_of_the_day']}, Delay: {entry['delay']} minutes."
        for entry in results
    ]
)

# Generate a prompt for Gemini
prompt = f"""
The user asked: {user_query}
Here are some relevant data points:
{context}

Please interpret the data and provide a summary in a conversational tone.
"""

# Use Gemini to generate a response
response = model.generate_content(prompt)
print(response.text)

Hmm, this data is a little strange! It looks like there are some seriously huge negative delays, especially on Tuesday evening. A negative delay means things are way ahead of schedule.  1377 minutes is over 22 hours! 

There's not enough information here to say why delays might be longer on Wednesday afternoons.  

Here's what we need to figure this out:

* **More Data:** We need data from many Wednesdays, not just one, and ideally from other times on Wednesday too. 
* **Positive Delays:** Negative delays this large are likely errors. We need to look at data that represents actual delays (positive numbers).
* **Context:** What are these delays for? Flights, trains, buses? Knowing the context helps us understand potential reasons for delays. 

Let's get some better data, and then we can try to solve this mystery! 



In [30]:
# User's input
user_query = "What delays should we expect on Saturday nights next week, and how can we minimize them?"

# Convert query to embedding
query_vector = sent_trans.encode(user_query).tolist()
query_vector.append(0.0)

# Query Pinecone
response = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)

# Extract metadata
results = [
    {
        "day": match['metadata']['day'],
        "time_of_the_day": match['metadata']['time_of_the_day'],
        "delay": match['metadata']['delay'],
        "score": match['score']
    }
    for match in response['matches']
]


# Format the context for Gemini
context = "\n".join(
    [
        f"Day: {entry['day']}, Time: {entry['time_of_the_day']}, Delay: {entry['delay']} minutes."
        for entry in results
    ]
)

# Generate a prompt for Gemini
prompt = f"""
The user asked: {user_query}
Here are some relevant data points:
{context}

Please interpret the data and provide a summary in a conversational tone.
"""

# Use Gemini to generate a response
response = model.generate_content(prompt)
print(response.text)

It looks like you're actually consistently EARLY on Saturday nights! Those negative delays mean you're arriving ahead of schedule by an average of about 31 minutes. 

So, good news! No need to worry about minimizing delays. You seem to have Saturday nights under control! 😉  

Do you want to check data for any other night of the week? 



In [31]:
# User's input
user_query = "How do Monday night delays compare to other nights of the week?"

# Convert query to embedding
query_vector = sent_trans.encode(user_query).tolist()
query_vector.append(0.0)

# Query Pinecone
response = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)

# Extract metadata
results = [
    {
        "day": match['metadata']['day'],
        "time_of_the_day": match['metadata']['time_of_the_day'],
        "delay": match['metadata']['delay'],
        "score": match['score']
    }
    for match in response['matches']
]


# Format the context for Gemini
context = "\n".join(
    [
        f"Day: {entry['day']}, Time: {entry['time_of_the_day']}, Delay: {entry['delay']} minutes."
        for entry in results
    ]
)

# Generate a prompt for Gemini
prompt = f"""
The user asked: {user_query}
Here are some relevant data points:
{context}

Please interpret the data and provide a summary in a conversational tone.
"""

# Use Gemini to generate a response
response = model.generate_content(prompt)
print(response.text)

It looks like Monday nights are running super smoothly!  ✈️

The data shows an average delay of **-16 minutes** for Monday nights. That means flights are actually taking off and landing **16 minutes earlier** than scheduled. 

However, we need more information to compare Monday nights to other nights of the week.  To give you a complete picture, I need data on the average delays for Tuesday, Wednesday, Thursday, Friday, Saturday, and Sunday nights too.  

Once you give me that info, I can tell you which night has the shortest (or longest!) delays. 😉 



In [34]:
# User's input
user_query = "What are the typical delays on Monday nights?"

# Convert query to embedding
query_vector = sent_trans.encode(user_query).tolist()
query_vector.append(0.0)

# Query Pinecone
response = index.query(
    vector=query_vector,
    top_k=20,
    include_metadata=True
)

# Extract metadata
results = [
    {
        "day": match['metadata']['day'],
        "time_of_the_day": match['metadata']['time_of_the_day'],
        "delay": match['metadata']['delay'],
        "score": match['score']
    }
    for match in response['matches']
]


# Format the context for Gemini
context = "\n".join(
    [
        f"Day: {entry['day']}, Time: {entry['time_of_the_day']}, Delay: {entry['delay']} minutes."
        for entry in results
    ]
)

# Generate a prompt for Gemini
prompt = f"""
The user asked: {user_query}
Here are some relevant data points:
{context}

Please interpret the data based on the user query and provide a summary in a conversational tone.
"""

# Use Gemini to generate a response
response = model.generate_content(prompt)
print(response.text)

It looks like you're interested in delays on Monday nights!  The data shows that Monday nights typically have very short delays, averaging around **4 to 16 minutes early**.  

However, there's something strange going on with the Tuesday evening data. Those delays are showing as almost -1400 minutes, which translates to nearly a full day early! That doesn't sound like a typical delay. It's more likely an error in the data or perhaps a cancellation. 

Let me know if you want to explore delays on other days or times, I'm happy to help! 😊 

